In [1]:
import pandas as pd
import numpy as np

In [4]:
food_waste=pd.read_csv("./data_preprocessing/1_food_waste_preprocessing.csv")
food_waste_sort = food_waste.sort_values(by=['emd_cd','base_date'] ,ascending=True)
food_waste_sort

,base_date,emd_cd,em_cnt,em_g
2019-11-30,2019-11-30,50110250,620,2279550
2019-12-31,2019-12-31,50110250,8178,31493650
2020-01-31,2020-01-31,50110250,13042,49632850
2020-02-29,2020-02-29,50110250,12264,46197050
2020-03-31,2020-03-31,50110250,14316,52561750
...,...,...,...,...
2021-02-28,2021-02-28,50130620,4563,15566250
2021-03-31,2021-03-31,50130620,4920,17300600
2021-04-30,2021-04-30,50130620,5256,17646850
2021-05-31,2021-05-31,50130620,6346,19494750


In [10]:
weather2019=pd.read_csv("./데이터 전처리/weather_2019v2.csv")
weather2018=pd.read_csv("./데이터 전처리/weather_2018v2.csv")
weather2019

,base_date,emd_cd,temp,rain
2019-01-31,2019-01-31,50110250.0,135.8,24.5
2019-02-28,2019-02-28,50110250.0,167.3,32.0
2019-03-31,2019-03-31,50110250.0,266.1,47.5
2019-04-30,2019-04-30,50110250.0,391.8,39.0
2019-05-31,2019-05-31,50110250.0,544.2,245.0
...,...,...,...,...
2019-08-31,2019-08-31,50130620.0,820.6,337.5
2019-09-30,2019-09-30,50130620.0,714.3,342.0
2019-10-31,2019-10-31,50130620.0,590.6,164.0
2019-11-30,2019-11-30,50130620.0,431.5,34.5


In [14]:
food_waste_sort_2019=food_waste_sort.merge(weather2019, on=['base_date','emd_cd'],how='inner')
food_waste_sort_2018=food_waste_sort.merge(weather2018,on=['base_date','emd_cd'],how='inner')
food_waste_sort_201819=pd.concat([food_waste_sort_2018, food_waste_sort_2019],ignore_index=True)
food_waste_sort_201819

,base_date,emd_cd,em_cnt,em_g,temp,rain
0,2018-01-31,50110253,3702,7744200,-3.122581,135.5
1,2018-02-28,50110253,3182,6799950,-1.925000,108.0
2,2018-03-31,50110253,3789,8239050,6.477419,350.0
3,2018-04-30,50110253,3733,7756850,9.756667,553.5
4,2018-05-31,50110253,4084,7819200,14.635484,402.0
...,...,...,...,...,...,...
868,2019-08-31,50130620,5903,20235490,820.600000,337.5
869,2019-09-30,50130620,5123,16327346,714.300000,342.0
870,2019-10-31,50130620,4794,15295791,590.600000,164.0
871,2019-11-30,50130620,4293,14201095,431.500000,34.5


In [24]:
# 회귀분석 시작~!
import statsmodels.api as sm

food_waste_sort_201819_data = food_waste_sort_201819.drop(['em_g'],axis=1)   # Feature에서 Target Data 분리

em_g = food_waste_sort_201819[['em_g']] # Target 변수저장
x_rain = food_waste_sort_201819['rain']


x_data.head()

# 회귀분석을 하기 위한 B_0, 상수항 추가
x_data1 = sm.add_constant(x_rain, has_constant = "add")

# 회귀모델 적합
multi_model = sm.OLS(em_g, x_rain)
fitted_multi_model = multi_model.fit()

# summary함수를 통해 OLS 결과 출력
fitted_multi_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                   em_g   R-squared (uncentered):                   0.327
Model:                            OLS   Adj. R-squared (uncentered):              0.327
Method:                 Least Squares   F-statistic:                              424.6
Date:                Thu, 09 Sep 2021   Prob (F-statistic):                    3.62e-77
Time:                        23:44:11   Log-Likelihood:                         -17068.
No. Observations:                 873   AIC:                                  3.414e+04
Df Residuals:                     872   BIC:                                  3.414e+04
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
rain        1.603e+05   7780.776     20.605      0.000    1.45e+05    1.76e+05
==============================================================================
Omnibus:                      138.458   Durbin-Watson:                   0.392
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              260.269
Skew:                           0.948   Prob(JB):                     3.04e-57
Kurtosis:                       4.888   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [22]:
corr=food_waste_sort_201819.corr(method='pearson')
corr

,emd_cd,em_cnt,em_g,temp,rain
emd_cd,1.000000,-0.451129,-0.382820,0.053998,-0.070709
em_cnt,-0.451129,1.000000,0.986177,-0.004420,0.173686
em_g,-0.382820,0.986177,1.000000,-0.022951,0.129840
temp,0.053998,-0.004420,-0.022951,1.000000,0.333577
rain,-0.070709,0.173686,0.129840,0.333577,1.000000
